In [57]:
import pandas as pd
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler,  Imputer, LabelBinarizer, OneHotEncoder,StandardScaler
from sklearn.feature_extraction import DictVectorizer

class LabelEncoderPipelineFriendly(LabelEncoder):
    
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelEncoderPipelineFriendly, self).fit(X)
        
    def transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).transform(X).reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).fit(X).transform(X).reshape(-1, 1)

In [58]:
df_train=pd.read_csv('E:\\Kaggle\\dont-overfit-ii\\train.csv')
df_test=pd.read_csv('E:\\Kaggle\\dont-overfit-ii\\test.csv')

In [59]:
def get_num_cols(df):
    colum_ind=[]
    for i in range(300):
        colum_ind.append(str(i))
    return df[colum_ind]

In [60]:
vec = make_union(*[
    make_pipeline(FunctionTransformer(get_num_cols, validate=False),StandardScaler() ),
])

In [61]:
x_train = vec.fit_transform(df_train)
x_test = vec.transform(df_test)
y_train = df_train['target']

In [62]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [64]:
#1.120,1.121,1.122,1.123,1.124,1.125,1.126,1.127,1.128,1.129
#0.085,0.4510511,0.4520512,0.4530513,0.4540514,0.4550515,0.4560516,0.4570517,0.4580518,0.4590519
#0.5,0.120,0.12121,0.123122,0.124123,0.125124,0.126125,0.127126,0.128127,0.129128
parameter_grid = {'alpha' : [0.085,0.4510511,0.4520512,0.4530513,0.4540514,0.4550515,0.4560516,0.4570517,0.4580518,0.4590519],
                  'l1_ratio' : [0.5,0.005,0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014],
                  
        
                 }

grid_search = GridSearchCV(en, param_grid=parameter_grid, cv=folds, scoring='roc_auc')
grid_search.fit(x_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.782638888888889
Best parameters: {'alpha': 0.085, 'l1_ratio': 0.5}


In [65]:
#en = ElasticNet(alpha=0.4520512, l1_ratio=1.12)
en = ElasticNet(alpha=0.085, l1_ratio=0.5)
en.fit(x_train, y_train)

ElasticNet(alpha=0.085, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [66]:
y_test=en.predict(x_test)

In [67]:
df_predicted = pd.DataFrame({'id': df_test['id'], 'target': y_test})
df_predicted.to_csv('E:\\Kaggle\\dont-overfit-ii\\sample_submission_std4.csv', sep=',', index=False)